In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from math import floor

In [2]:
img = cv2.imread("img1.png",0)
%matplotlib inline


In [20]:
def affine_def_matrix(λ,ψ,θ): #λ is zoom parameter-- ψ is rotation around axis-- θ is the angle between the normal
    θ=math.radians(θ)
    t=math.cos(θ)
    ψ=math.radians(ψ)
    affine_matrix=np.array([[λ*(math.cos(ψ)*t),λ*math.sin(ψ)],[λ*(-math.sin(ψ))*t,λ*(math.cos(ψ))]])
    
    return affine_matrix

def homography_matrix(warped_img,ref_img,affine):#Returns Inverse Homography Matrix
    m1=np.array([[1,0,len(warped_img)/2],[0,1,len(warped_img[0])/2],[0,0,1]])
    m2=np.array([[affine[0][0],affine[0][1],0],[affine[1][0],affine[1][1],0],[0,0,1]])
    m3=np.array([[1,0,-len(ref_img)/2],[0,1,-len(ref_img[0])/2],[0,0,1]])

    multiplication=np.dot(m1,m2)
    homography=np.dot(multiplication,m3)
    
    inverse_homography = np.linalg.inv(homography)
    
    return inverse_homography
    
affine_matrix=affine_def_matrix(0.5,30,50)

top_left=np.array([[0],[0]])
top_right=np.array([[0],[800]])
bottom_left=np.array([[640],[0]])
bottom_right=np.array([[640],[800]])

warped_top=np.dot(affine_matrix,top_left)
warped_right=np.dot(affine_matrix,top_right)
warped_left=np.dot(affine_matrix,bottom_left)
warped_bottom=np.dot(affine_matrix,bottom_right)

height=int(float(warped_bottom[0] - warped_top[0]))
width=int(float(warped_right[1]-warped_left[1]))

warped_img = np.zeros((height,width,1))

inv_homography=homography_matrix(warped_img,img,affine_matrix)

def bilinear_interpolation(x,y,ref_img):
    
    round1=round(float(x))
    round2=round(float(y))
    
    alpha=abs(round1-x)
    beta=abs(round2-y)
    
    if(round1>= len(ref_img) or round2 >= len(ref_img[0]) or round1<=0 or round2<=0):
     
        return 0
    else:
        
        if(x<round1):
            if(y<=round2 ):
                if(round2-1 >= len(ref_img[0]) and round1-1 >=len(ref_img)):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)
                        return int(w)
                elif(round2-1 >= len(ref_img[0]) and (round1-1 <len(ref_img))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1-1][round2]*alpha*(1-beta)
                        return int(w)
                elif(round1-1 >=len(ref_img) and (round2-1 < len(ref_img[0]))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2-1]*beta*(1-alpha)
                        return int(w)
                else:
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1-1][round2-1]*alpha*beta+ref_img[round1-1][round2]*alpha*(1-beta)+ref_img[round1][round2-1]*beta*(1-alpha)
                        return int(w)
            elif(y>round2):
                if(round2+1 >= len(ref_img[0]) and round1-1 >=len(ref_img)):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)
                        return int(w)
                elif(round2+1 >= len(ref_img[0]) and (round1-1 <len(ref_img))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1-1][round2]*alpha*(1-beta)
                        return int(w)
                elif(round1-1 >=len(ref_img) and (round2+1 < len(ref_img[0]))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2+1]*beta*(1-alpha)
                        return int(w) 
                else:
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2+1]*beta*(1-alpha)+ref_img[round1-1][round2+1]*alpha*beta+ref_img[round1-1][round2]*alpha*(1-beta)
                        return int(w)
        elif(x>round1):
            if(y<=round2 ):
                if(round2-1 >= len(ref_img[0]) and round1+1 >=len(ref_img)):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)
                        return int(w)
                elif(round2-1 >= len(ref_img[0]) and (round1+1 <len(ref_img))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1+1][round2]*alpha*(1-beta)
                        return int(w)
                elif(round1+1 >=len(ref_img) and (round2-1 < len(ref_img[0]))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2-1]*beta*(1-alpha)
                        return int(w)

                else:
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2-1]*beta*(1-alpha)+ref_img[round1+1][round2-1]*beta*alpha+ref_img[round1+1][round2]*alpha*(1-beta)
                        return int(w)
            elif(y>round2):
                
                if(round2+1 >= len(ref_img[0]) and round1+1 >=len(ref_img)):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)
                        return int(w)
                elif(round2+1 >= len(ref_img[0]) and (round1+1 <len(ref_img))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1+1][round2]*alpha*(1-beta)
                        return int(w)
                elif(round1+1 >=len(ref_img) and (round2+1 < len(ref_img[0]))):
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2+1]*beta*(1-alpha)
                        return int(w)
                
                else:
                        w=ref_img[round1][round2]*(1-alpha)*(1-beta)+ref_img[round1][round2+1]*beta*(1-alpha)+ref_img[round1+1][round2]*alpha*(1-beta)+ref_img[round1+1][round2+1]*alpha*beta
                        return int(w)
                
                
                
def apply_homography(ref_img,warped_img,inverse_homography):
    i=0
    while(i < len(warped_img)):
        j = 0
        while(j < len(warped_img[0])):
            homogeneous=np.array([[i],[j],[1]])
            p=np.dot(inverse_homography,homogeneous)
            
            warped_img[i][j]=bilinear_interpolation(p[0],p[1],ref_img)   
            j+=1
        i+=1
        
    return warped_img
    
i=apply_homography(img,warped_img,inv_homography)
cv2.imwrite("warped_img.png",i)   



[[ 0.2783352  0.25     ]
 [-0.1606969  0.4330127]]
0.278335199613
[[  2.78335200e-01   2.50000000e-01  -6.72638762e-02]
 [ -1.60696902e-01   4.33012702e-01   1.02717928e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


True

COMMENT-1:
We are multiply the affine matrix left side and its right side. This multiplication dislocates the image
and the result of this multiplication depends on sizes of the two images.The proper result of Left matrix and right matrix provides to fit the warped image.
    
COMMENT-2:
We apply this transformation operation to reach the intensity of reference image.Firstly,warped image is black.We don not know intensities of this.So we turned to the reference image with inverse homography to fill the warped image.


COMMENT-3:
We are using the bilinear interpolation because after the operations which explained in COMMENT-2,we find points and that points should not be integer and it is not match rightly to any pixel.So we rounds that points to reach a pixel and after that, we apply bilinear interpolation to get .We can choose the intensity of pixel which we rounded but this is not reflect the truth so getting average weight of the nearest 4 pixels with interpolation is reflect the truth better.
    
    